In [ ]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 文件路径和目标列
TARGET_COLUMN = 'GrainYield'
FEATURE_FILE = os.path.join("E:\\", "git", "phenomics", "barley", "project", "uav", "data", "threeD_dataset_mean.csv")
TARGET_FILE = r'E:\git\phenomics\barley\project\uav\data\agronomic\Agronomic_data_final.xlsx'

# 打印路径以调试
print("FEATURE_FILE:", FEATURE_FILE)
print("TARGET_FILE:", TARGET_FILE)

# 检查文件是否存在
if not os.path.exists(FEATURE_FILE):
    raise FileNotFoundError(f"Feature file not found: {FEATURE_FILE}")
if not os.path.exists(TARGET_FILE):
    raise FileNotFoundError(f"Target file not found: {TARGET_FILE}")

# 1. 加载数据
X = pd.read_csv(FEATURE_FILE)
target_df = pd.read_excel(TARGET_FILE)
y = target_df[TARGET_COLUMN]

# 删除指定行（索引173和218，对应第174和219行）
rows_to_drop = [173, 218]
X = X.drop(rows_to_drop)
y = y.drop(rows_to_drop)

# 检查目标变量中是否有0值
if 0 in y.values:
    print("目标变量中存在0值")

FEATURE_FILE: E:\git\phenomics\barley\project\uav\data\threeD_dataset_mean.csv
TARGET_FILE: E:\git\phenomics\barley\project\uav\data\agronomic\Agronomic_data_final.xlsx


In [46]:
# 2. 计算特征之间的相关性
correlation_matrix = X.drop('date', axis=1, errors='ignore').corr()  # errors='ignore' 防止date列不存在时出错
print("Correlation matrix shape:", correlation_matrix.shape)

# 找出相关性绝对值大于0.98的特征对
correlation_pairs = []
for i in range(correlation_matrix.shape[0]):
    for j in range(i + 1, correlation_matrix.shape[1]):
        if abs(correlation_matrix.iloc[i, j]) > 0.98:
            correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

print(f"\nFound {len(correlation_pairs)} pairs with correlation > 0.98 or < -0.98:")
for pair in correlation_pairs:
    print(f"{pair[0]} and {pair[1]}: {pair[2]:.4f}")

# 保存相关性对到文件
output_file = "correlation_pairs.txt"
with open(output_file, 'w') as f:
    f.write(f"Found {len(correlation_pairs)} pairs with correlation > 0.98 or < -0.98:\n")
    for pair in correlation_pairs:
        f.write(f"{pair[0]} and {pair[1]}: {pair[2]:.4f}\n")
print(f"Correlation pairs saved to {output_file}")



Correlation matrix shape: (62, 62)

Found 124 pairs with correlation > 0.98 or < -0.98:
NDVI_mean_msi and MTVI2_mean_msi: 0.9860
NDVI_mean_msi and OSAVI_mean_msi: 0.9934
NDVI_mean_msi and TVI_mean_msi: 0.9995
NDVI_mean_msi and GARI_mean_msi: 0.9973
NDVI_mean_msi and NAVI_mean_msi: 0.9964
NDVI_mean_msi and RGBVI_mean_msi: 0.9859
EVI_mean_msi and SAVI2_mean_msi: 0.9950
EVI_mean_msi and EVI2_mean_msi: 0.9948
EVI_mean_msi and MSAVI_mean_msi: 0.9947
EVI_mean_msi and MTVI2_mean_msi: 0.9907
EVI_mean_msi and OSAVI_mean_msi: 0.9885
EVI_mean_msi and MCARI1_mean_msi: 0.9921
EVI_mean_msi and MCARI2_mean_msi: 0.9980
EVI_mean_msi and RDVI_mean_msi: 0.9930
EVI_mean_msi and SAVI_mean_msi: 0.9950
NDWI_mean_msi and GNDVI_mean_msi: -1.0000
NDWI_mean_msi and BNDVI_mean_msi: -0.9872
SAVI2_mean_msi and EVI2_mean_msi: 0.9989
SAVI2_mean_msi and MSAVI_mean_msi: 0.9979
SAVI2_mean_msi and MTVI2_mean_msi: 0.9846
SAVI2_mean_msi and OSAVI_mean_msi: 0.9930
SAVI2_mean_msi and MCARI1_mean_msi: 0.9919
SAVI2_mean_msi an

In [49]:
from collections import defaultdict

# 设置优先级
feature_priorities = {'NDVI_mean_msi': 5.0}

def remove_highly_correlated_features(correlation_pairs, threshold, feature_priorities=None):
    if feature_priorities is None:
        feature_priorities = {}
    
    # 筛选高相关性对
    high_corr_pairs = [(pair[0], pair[1], pair[2]) for pair in correlation_pairs if abs(pair[2]) > threshold]
    if not high_corr_pairs:
        print("No feature pairs with correlation above the threshold.")
        return [], set()

    # 统计特征出现频率
    feature_freq = defaultdict(int)
    for f1, f2, _ in high_corr_pairs:
        feature_freq[f1] += 1
        feature_freq[f2] += 1

    all_features = set(feature_freq.keys())
    features_to_drop = set()

    # 调试：打印初始的高相关性对
    print("Initial high correlation pairs:", len(high_corr_pairs))

    while high_corr_pairs:
        max_score = -1
        feature_to_remove = None
        for feature in feature_freq:
            if feature in features_to_drop:
                continue
            freq = feature_freq[feature]
            priority = feature_priorities.get(feature, 0.5)
            score = freq / (priority + 1e-6)
            if score > max_score:
                max_score = score
                feature_to_remove = feature
        
        if feature_to_remove is None:
            print("No feature to remove, breaking loop.")
            break
        
        features_to_drop.add(feature_to_remove)
        print(f"Removing feature: {feature_to_remove} (freq: {feature_freq[feature_to_remove]}, priority: {priority}, score: {max_score:.4f})")
        
        # 更新高相关性对
        new_high_corr_pairs = [(f1, f2, corr) for f1, f2, corr in high_corr_pairs 
                               if f1 != feature_to_remove and f2 != feature_to_remove]
        high_corr_pairs = new_high_corr_pairs
        
        # 更新频率
        feature_freq = defaultdict(int)
        for f1, f2, _ in high_corr_pairs:
            feature_freq[f1] += 1
            feature_freq[f2] += 1

    features_to_keep = all_features - features_to_drop
    return list(features_to_drop), features_to_keep
# 假设 correlation_pairs 已正确定义，例如从前面的代码生成
# correlation_pairs = [('feat1', 'feat2', 0.99), ('feat2', 'feat3', 0.995), ...]
# 运行特征移除算法
features_to_drop, features_to_keep = remove_highly_correlated_features(
    correlation_pairs, threshold=0.99, feature_priorities=feature_priorities
)
print("\nFeatures to drop:", features_to_drop)
print("Features to keep:", features_to_keep)

Initial high correlation pairs: 66
Removing feature: EVI_mean_msi (freq: 8, priority: 0.5, score: 16.0000)
Removing feature: OSAVI_mean_msi (freq: 7, priority: 0.5, score: 14.0000)
Removing feature: SAVI2_mean_msi (freq: 6, priority: 0.5, score: 12.0000)
Removing feature: OSAVI-REG_mean_msi (freq: 6, priority: 0.5, score: 12.0000)
Removing feature: EVI2_mean_msi (freq: 5, priority: 0.5, score: 10.0000)
Removing feature: RDVI_mean_msi (freq: 4, priority: 0.5, score: 8.0000)
Removing feature: TVI_mean_msi (freq: 3, priority: 0.5, score: 6.0000)
Removing feature: NDRE_mean_msi (freq: 3, priority: 0.5, score: 6.0000)
Removing feature: SAVI_mean_msi (freq: 3, priority: 0.5, score: 6.0000)
Removing feature: CIredE_mean_msi (freq: 2, priority: 0.5, score: 4.0000)
Removing feature: NGRDI_mean_msi (freq: 2, priority: 0.5, score: 4.0000)
Removing feature: GCC_mean_rgb (freq: 2, priority: 0.5, score: 4.0000)
Removing feature: GARI_mean_msi (freq: 1, priority: 0.5, score: 2.0000)
Removing feature:

In [50]:
# 4. 筛选特征
X = X[list(features_to_keep)]
print(f"Remaining features after correlation filtering: {X.shape[1]}")

# 5. 数据预处理
imputer = KNNImputer(n_neighbors=5)
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 6. 数据分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. 定义和训练多个模型
models = {
    "Linear Regression": LinearRegression(),
    "SVR": SVR(kernel='rbf'),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(200,), max_iter=500, random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R²": r2}
    print(f"\n{name} Performance:")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"R-squared (R²): {r2:.4f}")

# 8. 可视化最佳模型（以XGBoost为例）
best_model = models["XGBoost"]
y_pred = best_model.predict(X_test)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('XGBoost: True vs Predicted Values')
plt.grid(True)
plt.tight_layout()
plt.show()

Remaining features after correlation filtering: 15


ValueError: Found input variables with inconsistent numbers of samples: [1150, 286]